# Abcd vs abcD


### Modules

In [1]:
#import os
#import numpy as np
import sympy as sp
sp.init_printing(use_latex='mathjax')
#import pandas as pd
#import ast
import itertools as itr

#from multiprocessing import Pool, cpu_count
#from tqdm.notebook import tqdm

from snappy import *
S = twister.Surface('S_2_1')

## Symplectic representation

In [2]:
K = sp.MatrixSymbol('K',2,2) #([[0,-1],[0,0]])
L = sp.MatrixSymbol('L',2,2) #([[1,0],[1,1]])
I = sp.Identity(2)
O = sp.ZeroMatrix(2,2)
J = sp.MatrixSymbol('J',2,2) #([[0,1],[-1,0]])

Ma = sp.BlockMatrix([[L.inv(), O], [O, I]])
Mb = sp.BlockMatrix([[L.transpose(), K], [K, L.transpose()]])
Mc = sp.BlockMatrix([[I, O], [O, L.inv()]])
Md = sp.BlockMatrix([[I, O], [O, L.transpose()]])
#display(Ma, Mb, Mc, Md)

MJ = sp.BlockMatrix([[J, O], [O, J]])
#display(MJ)

### Functions

In [3]:
def ev(M):
    return M.subs([
        (L, sp.Matrix([[1,0],[1,1]])),
        (K, sp.Matrix([[0,-1],[0,0]])),
        (J, sp.Matrix([[0,1],[-1,0]])),
    ]).as_explicit()
#evMd = ev(Md); display(evMd, evMd.det())

def is_Sp(M):
    if type(M) == sp.matrices.expressions.blockmatrix.BlockMatrix:
        return sp.block_collapse(ev(M).transpose()*ev(MJ)*ev(M)) == ev(MJ)
    else:
        return M.transpose()*ev(MJ)*M == ev(MJ)

for M in {Ma, Mb, Mc, Md}:
    if is_Sp(M):
        print(f"{M} is in Sp(4,Z)")

Matrix([
[L.T,   K],
[  K, L.T]]) is in Sp(4,Z)
Matrix([
[I,   0],
[0, L.T]]) is in Sp(4,Z)
Matrix([
[I,       0],
[0, L**(-1)]]) is in Sp(4,Z)
Matrix([
[L**(-1), 0],
[      0, I]]) is in Sp(4,Z)


### The symplectic images of Abcd and abcD

In [4]:
M_Abcd = sp.block_collapse((Ma.inv())*Mb*Mc*Md)
M1 = ev(M_Abcd)
display(M_Abcd, M1)
M_abcD = sp.block_collapse(Ma*Mb*Mc*(Md.inv()))
M2 = ev(M_abcD)
display(M_abcD, M2)

for M in {M1,M2}:
    print(f"is_Sp? --> {is_Sp(M)}")

⎡   T       -1  T⎤
⎢L⋅L   L⋅K⋅L  ⋅L ⎥
⎢                ⎥
⎢       T  -1  T ⎥
⎣ K    L ⋅L  ⋅L  ⎦

⎡1  1   1   0⎤
⎢            ⎥
⎢1  2   1   0⎥
⎢            ⎥
⎢0  -1  0   1⎥
⎢            ⎥
⎣0  0   -1  0⎦

⎡                      -1⎤
⎢ -1  T   -1    -1 ⎛ T⎞  ⎥
⎢L  ⋅L   L  ⋅K⋅L  ⋅⎝L ⎠  ⎥
⎢                        ⎥
⎢                    -1  ⎥
⎢          T  -1 ⎛ T⎞    ⎥
⎣  K      L ⋅L  ⋅⎝L ⎠    ⎦

⎡1   1   1   -2⎤
⎢              ⎥
⎢-1  0   -1  2 ⎥
⎢              ⎥
⎢0   -1  0   1 ⎥
⎢              ⎥
⎣0   0   -1  2 ⎦

is_Sp? --> True
is_Sp? --> True


### The charactoristic polynomial of M1 (M2)

In [5]:
cp_M1 = M1.charpoly().as_expr()
cp_M2 = M2.charpoly().as_expr()

print(f"cp_M1 == cp_M2 ?   --> {cp_M1==cp_M2}")

display(cp_M1.factor())

cp_M1 == cp_M2 ?   --> True


 4      3      2          
λ  - 3⋅λ  + 3⋅λ  - 3⋅λ + 1

## Rational canonical form

### The rational form of M1 (M2)

In [7]:
C = sp.matrices.expressions.CompanionMatrix(M1.charpoly()).as_explicit()
display(C)
print(f"is_Sp? --> {is_Sp(C)}")

⎡0  0  0  -1⎤
⎢           ⎥
⎢1  0  0  3 ⎥
⎢           ⎥
⎢0  1  0  -3⎥
⎢           ⎥
⎣0  0  1  3 ⎦

is_Sp? --> False


### basis of the T-cyclic (sub)space

In [14]:
def transition_matrix(A,v):
    if type(v) == list:
        v = sp.Matrix(4,1,v)
    vecs = [(A**k)*v for k in range(5)]
    return sp.Matrix(4,4,lambda i,j: vecs[j][i])

In [ ]:
P = transition_matrix(M1,[0,-1,1,0])
Q = transition_matrix(M2,[1,0,0,0])

for X in [P,Q]:
    display(X, X.det(), is_Sp(X)) #, P.inv())

### The matrix giving conjugacy of M1 with M2

In [ ]:
S2 = Q*(P.inv())
display(S2, S2.det(), is_Sp(S2))

X = S2*M1*(S2.inv())
#display(X)
print(f"X==M2 ? --> {X==M2}")

### All_in_One

In [8]:
#### dAbc (M1)

e1 = sp.Matrix(4,1,[1,0,0,0])
vecs = [(M1**k)*e1 for k in range(4)]

P = sp.Matrix(4,4,lambda i,j: vecs[j][i])
display(P, P.det()) #, P.inv())
print(P*C*(P.inv()) == M1)

⎡1  1  2   4 ⎤
⎢            ⎥
⎢0  1  3   7 ⎥
⎢            ⎥
⎢0  0  -1  -3⎥
⎢            ⎥
⎣0  0  0   1 ⎦

-1

True


In [9]:
#### Dabc (M2)

e1 = sp.Matrix(4,1,[0,1,1,0])
vecs = [(M2**k)*e1 for k in range(4)]

Q = sp.Matrix(4,4,lambda i,j: vecs[j][i])
display(Q, Q.det()) #, Q.inv())
print(Q*C*(Q.inv()) == M2)

⎡0  2   2   1 ⎤
⎢             ⎥
⎢1  -1  -3  -4⎥
⎢             ⎥
⎢1  -1  0   2 ⎥
⎢             ⎥
⎣0  -1  -1  -2⎦

9

True


In [10]:
S1 = Q*(P.inv())
display(S1, S1.det())

⎡0  2   4   -1⎤
⎢             ⎥
⎢1  -2  -1  3 ⎥
⎢             ⎥
⎢1  -2  -4  0 ⎥
⎢             ⎥
⎣0  -1  -2  -1⎦

-9

In [11]:
X = S1*M1*(S1.inv())
display(X)

print(X==M2)

⎡1   1   1   -2⎤
⎢              ⎥
⎢-1  0   -1  2 ⎥
⎢              ⎥
⎢0   -1  0   1 ⎥
⎢              ⎥
⎣0   0   -1  2 ⎦

True


In [12]:
def get_matrix_connecting_M1_with_M2(v, w):
    P = transition_matrix(M1,v)
    Q = transition_matrix(M2,w)
    return (P, Q, P*(Q.inv()))

In [13]:
v, w = [0,1,0,0], [1,0,0,0] #[1,-1,0,0], [1,-1,0,-1] #[1,-1,0,0], [1,0,0,0] #
P, Q, S = get_matrix_connecting_M1_with_M2(v,w)
print(f"{v=}, {w=} \n\n--->  {(P.det(), Q.det())=}, {(is_Sp(P), is_Sp(Q))=}") 
print(S)
print(f"{is_Sp(S)=}")

display(S)

NameError: name 'transition_matrix' is not defined

In [ ]:
vects = [ list(x) for x in itr.product({-1,0,1}, repeat=4)]
vects.pop(0)
print(vects[0:6])

v = [x*(-1) for x in vects.pop(0)]
vects.remove(v)

print(vects[0:6])
print(vects[-1])

In [ ]:
row_vecs = [list(x) for x in itr.product({-1,0,1}, repeat=4)]
row_vecs.pop(0)
print(row_vecs[:12], "\n")

vecs = []
while len(row_vecs) > 0:
    vecs.append(row_vecs.pop(0))
    row_vecs.remove([x*(-1) for x in vecs[-1]])
    
s = len(vecs); print(vecs[:6])

In [ ]:
for v, w in itr.product(vecs, repeat=2):
    P, Q, S = get_matrix_connecting_M1_with_M2(v,w)
    if is_Sp(S): #P.det() == Q.det():
        print(f"{v=}, {w=}  --->  {(P.det(), Q.det())=}") 
        display(S)
        print(f"{is_Sp(S)=} \n -----\n")

In [ ]:
v, w = [0,-1,1,0], [1,0,0,0]
S, T, X = get_matrix_connecting_M1_with_M2(v, w)
print(f"{v=}, {w=}  --->  {(S.det(), T.det())=}") 
display(X)
print(f"{is_Sp(X)=} \n -----\n")

for v, w in itr.product(vecs, repeat=2):
    P, Q, S = get_matrix_connecting_M1_with_M2(v,w)
    if S == X: #P.det() == Q.det():
        print(f"{v=}, {w=}  --->  {(P.det(), Q.det())=}") 
#        display(S)
#        print(f"{is_Sp(S)=} \n -----\n")

## Data

v=[0, 1, 0, 0], w=[1, 0, 0, 0] 
--->  (P.det(), Q.det())=(1, 1)
Matrix([[-1, 1, 0, -2], [0, 0, 1, 1], [-1, 0, -1, 1], [-1, 0, -1, 0]])
is_Sp(S)=False

----
v=[1, -1, 0, 0], w=[1, 0, 0, 0] 
--->  (P.det(), Q.det())=(1, 1)
Matrix([[0, -1, 0, 1], [1, 1, 0, 0], [0, 0, 0, -1], [1, 1, 1, 0]])
is_Sp(S)=True

----
v=[0, 1, 0, 0], w=[1, -1, 0, -1] 
--->  (P.det(), Q.det())=(1, 1)
Matrix([[0, 1, 0, -1], [-1, -1, 0, 0], [0, 0, 0, 1], [-1, -1, -1, 0]])
is_Sp(S)=True

----
v=[1, -1, 0, -1], w=[1, -1, 0, -1] 
--->  (P.det(), Q.det())=(1, 1)
Matrix([[-1, -2, -2, 0], [2, 3, 2, 0], [0, 0, 1, 0], [2, 2, 2, 1]])
is_Sp(S)=True

----
v=[1, -1, 0, 0], w=[1, -1, 0, -1] 
--->  (P.det(), Q.det())=(1, 1)
Matrix([[-4, -5, -3, -1], [8, 9, 6, 3], [-1, -1, -1, -1], [6, 7, 5, 2]])
is_Sp(S)=False

In [ ]:
c = sp.symbols('c:4')
I4 = sp.BlockMatrix([[I, O], [O, I]]).as_explicit()

In [ ]:
d1 = (c[0]*I4 + c[1]*M1 + c[2]*(M1**2) + c[3]*(M1**3)).det()
d2 = (c[0]*I4 + c[1]*C + c[2]*(C**2) + c[3]*(C**3)).det()

d1 == d2

In [ ]:
d2

# Note for 12/9

In [ ]:
P, Q, X0 = get_matrix_connecting_M1_with_M2([0,-1,1,0],[1,0,0,0])
display(X0)
print(f"{(M2 == X0.inv()*M1*X0)=}, {is_Sp(X0)=}")

In [ ]:
v, w = [1,1,0,0], [0,1,1,1]
P, Q, X = get_matrix_connecting_M1_with_M2(v,w)

print(f"{(M2 == X.inv()*M1*X)=}, {is_Sp(X)=}")

print(f"{is_Sp(Q*(P.inv()))=}")

In [ ]:
def poly(v,A):
    terms = [v[i]*(A**i) for i in range(len(v))]
    return v[0]*(A**0)+v[1]*(A**1)+v[2]*(A**2)+v[3]*(A**3)

In [ ]:
p, q = poly([1,-1,1,1], M2), poly([1,1,0,-1], M2)
Y = X0*p*(q.inv())

display(Y)
print(f"{(M2 == Y.inv()*M1*Y)=},  {is_Sp(Y)=},  {is_Sp(p*(q.inv()))=}")

In [ ]:
row_vecs = [list(x) for x in itr.product({-1,0,1}, repeat=4)]
row_vecs.pop(0)
print(row_vecs[:12], "\n")

vecs = []
while len(row_vecs) > 0:
    vecs.append(row_vecs.pop(0))
    row_vecs.remove([x*(-1) for x in vecs[-1]])
    
s = len(vecs); print(vecs[:6])

for v, w in itr.product(vecs, repeat=2):
    if v == w:
        continue
    p, q = poly(v, M2), poly(w, M2)
    S = p*(q.inv())
    X = X0*S
    if is_Sp(S): #X == X0: #P.det() == Q.det():
        print(f"{v=}, {w=}  --->  {(p.det(), q.det())=}") 
        display(S, X)
        print(f"{is_Sp(X)=} \n -----\n")

In [ ]:
A = poly([0,1,0,0], M2)
B = poly([0,0,1,0], M2)

display(A,B)

In [ ]:
poly([0,1,1,0], M2)

In [ ]:
c = sp.symbols('c:4')
g = [c[i] for i in range(4)]

Cg = sp.Matrix(0,0,[])
for i in range(4):
    v = (C**i)*sp.Matrix(4,1,g)
    Cg = Cg.row_join(v)
    
display(Cg)

In [ ]:
D = Cg.det()
DCginv = D*Cg.inv()

In [ ]:
display(DCginv)

In [ ]:
d = sp.symbols('d:4')
h = sp.Matrix(4,1,[d[i] for i in range(4)])

display(Cg*h)

t = sp.symbols('t')
exp = (poly(c,t)*poly(d,t)).as_poly()

#display(exp.degree())
Q,R = exp.div((t**4).as_poly())
#display(Q,R)
exp = Q*f+R
display(exp)

In [ ]:
from sympy.polys import ring, ZZ
R, t = ring("t", ZZ)

R.dmp_trunc(exp, f)

In [ ]:
from sympy import reduced

reduced(exp, [poly([-1,3,-3,3],t)])

In [ ]:
#f = poly([-1,3,-3,3],t).as_poly()
display(f)
f.is_irreducible

In [ ]:
for trm in exp.all_terms():
    deg = trm[0][0]
    display((trm[1], t**deg))#*t**trm[0][0])